# Attention-Based Model for Tabular Features

**Rationale**: The paper's GNN achieved 0.0039 MSE using Graph Attention Networks (GAT).
We can approximate some of this with self-attention on tabular features.

**Approach**:
1. Self-attention on solvent features (like simplified GAT)
2. Learned mixture encoding (instead of linear interpolation)
3. Cross-attention between solvent A and B features for mixtures

**Expected outcome**: Long shot - unlikely to close 3x gap, but worth trying.

**Baseline to beat**: exp_012 CV 0.009004, LB 0.0913

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading
DATA_PATH = '/home/data'
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load features
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

Spange: (26, 13), DRFP filtered: (24, 122)


In [3]:
# Attention-based featurizer with learned mixture encoding
class AttentionFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        # Feature dimensions
        self.kinetic_dim = 5  # time, temp, 1/T, log(t), interaction
        self.spange_dim = self.spange_df.shape[1]  # 13
        self.drfp_dim = self.drfp_df.shape[1]  # 122
        self.total_dim = self.kinetic_dim + self.spange_dim + self.drfp_dim  # 140

    def featurize(self, X, return_separate=False):
        """Return features, optionally separated for attention."""
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if return_separate:
                # Return separate features for attention
                return {
                    'kinetic': torch.tensor(X_kinetic),
                    'A_spange': torch.tensor(A_spange),
                    'B_spange': torch.tensor(B_spange),
                    'A_drfp': torch.tensor(A_drfp),
                    'B_drfp': torch.tensor(B_drfp),
                    'pct': torch.tensor(pct)
                }
            else:
                # Linear interpolation (baseline)
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            
            if return_separate:
                return {
                    'kinetic': torch.tensor(X_kinetic),
                    'spange': torch.tensor(X_spange),
                    'drfp': torch.tensor(X_drfp)
                }
        
        return torch.tensor(np.hstack([X_kinetic, X_spange, X_drfp]))

print('AttentionFeaturizer defined')

AttentionFeaturizer defined


In [4]:
# Self-Attention Module
class SelfAttention(nn.Module):
    def __init__(self, input_dim, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = input_dim // num_heads
        assert input_dim % num_heads == 0, "input_dim must be divisible by num_heads"
        
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.out = nn.Linear(input_dim, input_dim)
        
    def forward(self, x):
        # x: (batch, seq_len, dim) or (batch, dim)
        if x.dim() == 2:
            x = x.unsqueeze(1)  # (batch, 1, dim)
        
        batch_size, seq_len, _ = x.shape
        
        Q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attn = F.softmax(scores, dim=-1)
        
        out = torch.matmul(attn, V)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        out = self.out(out)
        
        return out.squeeze(1) if seq_len == 1 else out

print('SelfAttention defined')

SelfAttention defined


In [5]:
# Attention-based MLP Model
class AttentionMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05, use_attention=True):
        super().__init__()
        self.use_attention = use_attention
        
        # Input projection to make divisible by 4 (for attention heads)
        self.proj_dim = ((input_dim // 4) + 1) * 4  # Round up to nearest multiple of 4
        self.input_proj = nn.Linear(input_dim, self.proj_dim)
        
        # Self-attention layer
        if use_attention:
            self.attention = SelfAttention(self.proj_dim, num_heads=4)
            self.attn_norm = nn.LayerNorm(self.proj_dim)
        
        # MLP layers
        layers = [nn.BatchNorm1d(self.proj_dim)]
        prev_dim = self.proj_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        # Project input
        x = self.input_proj(x)
        
        # Apply self-attention
        if self.use_attention:
            attn_out = self.attention(x)
            x = self.attn_norm(x + attn_out)  # Residual connection
        
        # MLP
        return self.mlp(x)

print('AttentionMLP defined')

AttentionMLP defined


In [6]:
# Attention Ensemble Model
class AttentionEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single', use_attention=True):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.use_attention = use_attention
        self.featurizer = AttentionFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize(X_train)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # For mixtures, also use flipped version
            X_train_flip = X_train.copy()
            X_train_flip['SOLVENT A NAME'] = X_train['SOLVENT B NAME']
            X_train_flip['SOLVENT B NAME'] = X_train['SOLVENT A NAME']
            X_train_flip['SolventB%'] = 1 - X_train['SolventB%']
            X_flip = self.featurizer.featurize(X_train_flip)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            model = AttentionMLP(input_dim, hidden_dims=self.hidden_dims, dropout=0.05, 
                                 use_attention=self.use_attention).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X).to(device)
            # Flipped version
            X_flip_df = X.copy()
            X_flip_df['SOLVENT A NAME'] = X['SOLVENT B NAME']
            X_flip_df['SOLVENT B NAME'] = X['SOLVENT A NAME']
            X_flip_df['SolventB%'] = 1 - X['SolventB%']
            X_flip = self.featurizer.featurize(X_flip_df).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += (model(X_std) + model(X_flip)) * 0.5
            return (pred_sum / self.n_models).cpu().numpy()
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            return (pred_sum / self.n_models).cpu().numpy()

print('AttentionEnsemble defined')

AttentionEnsemble defined


In [7]:
# Run CV for single solvent with attention model
print("\n--- TASK 0: Single Solvent (24 folds) - Attention Model ---")
X, Y = load_data("single_solvent")
all_preds_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y)), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    model = AttentionEnsemble(hidden_dims=[32, 16], n_models=5, data='single', use_attention=True)
    model.train_model(train_X, train_Y, epochs=200)
    predictions = model.predict(test_X)
    all_preds_single.append(predictions)
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_preds_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) - Attention Model ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [02:18<53:01, 138.33s/it]

  8%|▊         | 2/24 [04:45<52:37, 143.53s/it]

 12%|█▎        | 3/24 [07:02<49:09, 140.45s/it]

...

 92%|█████████▏| 22/24 [47:48<04:04, 122.17s/it]

 96%|█████████▌| 23/24 [49:40<01:59, 119.13s/it]

100%|██████████| 24/24 [51:26<00:00, 115.03s/it]

100%|██████████| 24/24 [51:26<00:00, 128.60s/it]


Single Solvent MSE: 0.035719


In [8]:
# Run CV for full data with attention model
print("\n--- TASK 1: Full Data (13 folds) - Attention Model ---")
X, Y = load_data("full")
all_preds_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X, Y)), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    model = AttentionEnsemble(hidden_dims=[32, 16], n_models=5, data='full', use_attention=True)
    model.train_model(train_X, train_Y, epochs=200)
    predictions = model.predict(test_X)
    all_preds_full.append(predictions)
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_preds_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) - Attention Model ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [05:56<1:11:14, 356.20s/it]

 15%|█▌        | 2/13 [11:43<1:04:22, 351.17s/it]

 23%|██▎       | 3/13 [17:39<58:50, 353.05s/it]  

...

 85%|████████▍ | 11/13 [1:07:40<12:23, 371.94s/it]

 92%|█████████▏| 12/13 [1:13:14<06:00, 360.64s/it]

100%|██████████| 13/13 [1:18:46<00:00, 351.96s/it]

100%|██████████| 13/13 [1:18:46<00:00, 363.61s/it]


Full Data MSE: 0.016747


In [9]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_012 (2-model ensemble): CV 0.009004, LB 0.0913 (BEST)')
print(f'This attention model: CV {overall_mse:.6f}')

if overall_mse < 0.009004:
    improvement = (0.009004 - overall_mse) / 0.009004 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_012!')
    if improvement > 10:
        print('SIGNIFICANT improvement - consider submitting!')
    else:
        print('Marginal improvement - may not translate to LB improvement.')
else:
    degradation = (overall_mse - 0.009004) / 0.009004 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_012')
    print('Keep exp_012 as best submission.')


=== FINAL RESULTS ===
Single Solvent MSE: 0.035719 (n=656)
Full Data MSE: 0.016747 (n=1227)
Overall MSE: 0.023357

=== COMPARISON ===
exp_012 (2-model ensemble): CV 0.009004, LB 0.0913 (BEST)
This attention model: CV 0.023357

✗ WORSE: 159.40% worse than exp_012
Keep exp_012 as best submission.
